In [1]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub

ImportError: Traceback (most recent call last):
  File "C:\Users\Tasnim\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
model = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1").signatures["default"]

In [ ]:

model2 = hub.load("D:\object detection pro\model files").signatures["default"]

In [ ]:
model3 = hub.load("D:\object detection pro\2nd model").signatures["default"]

In [4]:
import random
import math


In [ ]:

colorcodes={}

In [ ]:
def drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color):
    im_height, im_width, _  = image.shape
    left,top,right,bottom = int(xmin*im_width), int(ymin*im_height), int(xmax*im_width),int(ymax*im_height)
    cv2.rectangle(image,(left,top),(right,bottom),color = color,thickness = 4)
    FONT_SCALE = 5e-3
    THICKNESS_SCALE = 4e-3
    width = right-left
    height = bottom-top
    TEXT_Y_OFFSET_SCALE = 1e-2
    cv2.rectangle(
        image,
        (left,top- int(height * 6e-2)),
        (right,top),
        color = color,
        thickness = -1
        
    )
    cv2.putText(
        image,
        namewithscore,
        (left,top-int(height * TEXT_Y_OFFSET_SCALE)),
        fontFace = cv2.FONT_HERSHEY_PLAIN,
        fontScale = min(width,height)* FONT_SCALE,
        thickness = math.ceil(min(width,height)* THICKNESS_SCALE),
        color = (255,255,255)
    )

In [ ]:
def draw(image,boxes,classnames,scores):
    boxesidx = tf.image.non_max_suppression(boxes,scores,max_output_size = 4, iou_threshold = 0.5,score_threshold = 0.1)
#     for i in range(len(boxes)):
    for i in boxesidx:
        ymin,xmin,ymax,xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0,255,30)
            c2 = random.randrange(0,255,25)
            c3 = random.randrange(0,255,50)
            colorcodes.update({classname: (c1,c2,c3)})
            color = colorcodes[classname]
        namewithscore = "{}:{}".format(classname,int(100*scores[i]))
        drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color)
        
    return image

In [ ]:
image = cv2.imread("D:\object detection pro\image.jpg")
image = cv2.resize(image,(800,600))
image2 = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
converted_img = tf.image.convert_image_dtype(image2,tf.float32)[tf.newaxis, ...]

In [ ]:
detection = model2(converted_img)

In [ ]:
print(detection["detection_class_entities"])

In [ ]:
result = {key:value.numpy() for key,value in detection.items()}
imagewithboxes = draw(image,result['detection_boxes'],result['detection_class_entities'],result["detection_scores"])
cv2.imshow("image",imagewithboxes)
cv2.waitKey(0)

In [ ]:
image2 = cv2.imread("D:\object detection pro\image2.jpg")
image3 = cv2.cvtColor(image2,cv2.COLOR_BGR2RGB)
converted_img = tf.image.convert_image_dtype(image3,tf.float32)[tf.newaxis, ...]

In [ ]:
detection = model3(converted_img)

In [ ]:
result = {key:value.numpy() for key,value in detection.items()}
imagewithboxes = draw(image2,result['detection_boxes'],result['detection_class_entities'],result["detection_scores"])
cv2.imwrite("detected.jpg",imagewithboxes)

In [ ]:
image3 = cv2.imread("D:\object detection pro\image3.png")
image4 = cv2.cvtColor(image3,cv2.COLOR_BGR2RGB)
converted_img = tf.image.convert_image_dtype(image4,tf.float32)[tf.newaxis, ...]

In [ ]:
detection = model3(converted_img)

In [ ]:
result = {key:value.numpy() for key,value in detection.items()}
imagewithboxes = draw(image3,result['detection_boxes'],result['detection_class_entities'],result["detection_scores"])
cv2.imshow("image",imagewithboxes)
cv2.waitKey(0)